In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Diabetes study

**Index**

1. [Problem understanding](#Problem-understanding)
2. [Data understanding](#Data-understanding)
3. [Data preparation](#data-preparation)

<a id="problem-understanding"></a>
## Problem understanding

- Diabetes is a major contributor to risk for hospital readmission, representing nearly one-fifth of all 30-day unplanned hospital readmissions. (Soh et al. 2020)
- <u>Patient characteristics may affect the outcomes</u> (such as gender, age, race, and comorbidities) (Soh et al. 2020)
- Early hospital readmissions have been recognized as a common and costly occurrence, particularly among elderly and high risk patients. One in five Medicare beneficiaries is readmitted within 30 days, for example, at a cost of over $26 billion per year.1 To encourage improvement in the quality of care and a reduction in unnecessary health expense; policymakers, reimbursement strategists, and the United States government have made reducing 30-day hospital readmissions a national priority.2–4 Achieving this goal, however, requires more complete understanding of the underlying causes of readmission. https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4249925/

<u>**Objective**</u>

In this work I will try to:

- Create a predictive model for early readmission (i.e. unplanned readmissions that happen within 30 days of discharge from the initial admission). Such a model could help plan interventions for high-risk patients and reduce costs for both patients and hospitals.
- Point out patterns and insights about readmissions, and answer the following questions:
  - Which variables are the strongest readmission predictors?
  - Given that HbA1c values are key for planning the diabetic patient's medication (according to the literature), how often HbA1c exams are asked by physicians in the inpatient setting?


<a id="data-understanding"></a>
## Data understanding

The data set comprises two files, `diabetic_data.csv` and `IDs_mapping.csv`. The former contains anonymized medical data collected during 100k encounters accross several hospitals in the United States over a decade. The latter contains a legend for some of the numerical categories (like admission type and discharge disposition).

In [2]:
df = pd.read_csv("../data/diabetic_data.zip")
df.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


Missing values are represented by question marks in this data set. They must be replaced with np.nan if we want to use Pandas' default methods to view and handle missing data.

In [3]:
df.replace("?", np.nan, inplace=True)

Overral view of the dataframe, including missing values and dtypes:

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 50 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   encounter_id              101766 non-null  int64 
 1   patient_nbr               101766 non-null  int64 
 2   race                      99493 non-null   object
 3   gender                    101766 non-null  object
 4   age                       101766 non-null  object
 5   weight                    3197 non-null    object
 6   admission_type_id         101766 non-null  int64 
 7   discharge_disposition_id  101766 non-null  int64 
 8   admission_source_id       101766 non-null  int64 
 9   time_in_hospital          101766 non-null  int64 
 10  payer_code                61510 non-null   object
 11  medical_specialty         51817 non-null   object
 12  num_lab_procedures        101766 non-null  int64 
 13  num_procedures            101766 non-null  int64 
 14  num_

I need more details about possible values. Knowing the variable's dtypes only won't be enough, so let's see some examples of what to expect.

In [5]:
print("Variable              Unique     % Missing  Examples")
print("="*100)
for column in df:
    uniques = set(df[column])
    n_uniques = len(uniques)
    missing = df[column].isnull().mean()*100
    examples = list(uniques)[:5]
    output = "{:<25} {:<10} {:<6.1f} {:<20}".format(column, n_uniques, missing, str(examples))
    print(output)
    

Variable              Unique     % Missing  Examples
encounter_id              101766     0.0    [77856768, 173015040, 84934662, 273678342, 17563668]
patient_nbr               71518      0.0    [33947649, 92667906, 82706436, 83623941, 128319494]
race                      6          2.2    [nan, 'Hispanic', 'Asian', 'Other', 'Caucasian']
gender                    3          0.0    ['Female', 'Unknown/Invalid', 'Male']
age                       10         0.0    ['[60-70)', '[30-40)', '[70-80)', '[50-60)', '[10-20)']
weight                    10         96.9   [nan, '[125-150)', '[150-175)', '[25-50)', '[75-100)']
admission_type_id         8          0.0    [1, 2, 3, 4, 5]     
discharge_disposition_id  26         0.0    [1, 2, 3, 4, 5]     
admission_source_id       17         0.0    [1, 2, 3, 4, 5]     
time_in_hospital          14         0.0    [1, 2, 3, 4, 5]     
payer_code                18         39.6   [nan, 'OT', 'MC', 'HM', 'BC']
medical_specialty         73         49.1   [n

Evaluating the target variable balance:

In [6]:
df.readmitted.value_counts()

NO     54864
>30    35545
<30    11357
Name: readmitted, dtype: int64

Evaluating the number of patients to whom diabetes medication was prescribred during the inpatient settings:

In [7]:
drugs = ['metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone']
drugs_values = ["No", "Up", "Steady", "Down"]
df[drugs].apply(pd.value_counts).T

,Down,No,Steady,Up
metformin,575.0,81778.0,18346.0,1067.0
repaglinide,45.0,100227.0,1384.0,110.0
nateglinide,11.0,101063.0,668.0,24.0
chlorpropamide,1.0,101680.0,79.0,6.0
glimepiride,194.0,96575.0,4670.0,327.0
acetohexamide,NaN,101765.0,1.0,NaN
glipizide,560.0,89080.0,11356.0,770.0
glyburide,564.0,91116.0,9274.0,812.0
tolbutamide,NaN,101743.0,23.0,NaN
pioglitazone,118.0,94438.0,6976.0,234.0


### IDs Mapping

Now taking a look at `IDs_mapping.csv`, the file isn't in standard CSV format. It's actually a text file mapping numerical categories to strings. Using this file, I'll replace each number with the appropriate name in the original dataframe.

In [8]:
# Map for admission_type_id
admtype_map = pd.read_csv("../data/IDs_mapping.zip", skiprows=0, nrows=8)

# Map for discharge_disposition_id
discharge_map = pd.read_csv("../data/IDs_mapping.zip", skiprows=10, nrows=30)

# Map for admission_source_id
admsrc_map = pd.read_csv("../data/IDs_mapping.zip", skiprows=42, nrows=25)

In [9]:
admsrc_map.head()

,admission_source_id,description
0,1,Physician Referral
1,2,Clinic Referral
2,3,HMO Referral
3,4,Transfer from a hospital
4,5,Transfer from a Skilled Nursing Facility (SNF)


Digging a little further, I've found out that there's a research article by Strack et al. (2014), which is the group behind making the data set publicly available. In this article, they make a rundown of each category and what they mean:

- **Encounter ID** - Unique identifier of an encounter
- **Patient number** - Unique identifier of a patient
- **Race Values** - Values: Caucasian, Asian, African American, Hispanic, and other
- **Gender Values** - Values: male, female, and unknown/invalid
- **Age Grouped in 10-year intervals** - 0, 10), 10, 20), …, 90, 100)
- **Weight** - Weight in pounds
- **Admission type** - Integer identifier corresponding to 9 distinct values, for example, emergency, urgent, elective, newborn, and not available
- **Discharge disposition** - Integer identifier corresponding to 29 distinct values, for example, discharged to home, expired, and not available
- **Admission source** - Integer identifier corresponding to 21 distinct values, for example, physician referral, emergency room, and transfer from a hospital
- **Time in hospital** - Integer number of days between admission and discharge
- **Payer code** - Integer identifier corresponding to 23 distinct values, for example, Blue Cross/Blue Shield, Medicare, and self-pay Medical
- **Medical specialty** - Integer identifier of a specialty of the admitting physician, corresponding to 84 distinct values, for example, cardiology, internal medicine, family/general practice, and surgeon
- **Number of lab procedures** - Number of lab tests performed during the encounter
- **Number of procedures** - Numeric Number of procedures (other than lab tests) performed during the encounter
- **Number of medications** - Number of distinct generic names administered during the encounter
- **Number of outpatient visits** - Number of outpatient visits of the patient in the year preceding the encounter
- **Number of emergency visits** - Number of emergency visits of the patient in the year preceding the encounter
- **Number of inpatient visits** - Number of inpatient visits of the patient in the year preceding the encounter
- **Diagnosis 1** - The primary diagnosis (coded as first three digits of ICD9); 848 distinct values
- **Diagnosis 2** - Secondary diagnosis (coded as first three digits of ICD9); 923 distinct values
- **Diagnosis 3** - Additional secondary diagnosis (coded as first three digits of ICD9); 954 distinct values
- **Number of diagnoses** - Number of diagnoses entered to the system 0%
- **Glucose serum test result** - Indicates the range of the result or if the test was not taken. Values - “>200,” “>300,” “normal,” and “none” if not measured
- **A1c test result** - Indicates the range of the result or if the test was not taken. Values: “>8” if the result was greater than 8%, “>7” if the result was greater than 7% but less than 8%, “normal” if the result was less than 7%, and “none” if not measured.
- **Change of medications** - Indicates if there was a change in diabetic medications (either dosage or generic name). Values: “change” and “no change”
- **Diabetes medications** - Indicates if there was any diabetic medication prescribed. Values: “yes” and “no”
- **24 features for medications** - columns named after diabetes-related medication indicating whether the drug was prescribed or there was a change in dosage. Values: “up” if the dosage was increased during the encounter, “down” if the dosage was decreased, “steady” if the dosage did not change, and “no” if the drug was not prescribed
- **Readmitted** - Days to inpatient readmission. Values: “<30” if the patient was readmitted in less than 30 days, “>30” if the patient was readmitted in more than 30 days, and “No” for no record of readmission

### Data set characteristics

- Covers a 10-year span (1999-2008).
- Hospital admissions in the data set are supposed to be diabetic-related **only**.
- Lab tests were performed during admission.
- Medications were administered during admission.
- Data set contains <u>multiple readmissions of the same people</u>.
- Lots of low-variance variables in this data set (e.g. `examide` and `citoglipton` have only 1 unique value), which won't add to the model's predictive power.
- `diag_1` to `diag_3` columns have too many ICD-9-CM codes and will have to be shortened
- Missing values:
  - `weight` variable is nearly useless, with about 97% of missing values.
  - `payercode` could be an useful variable, but has a concerning number of missing values (around 40%)
  - `medical_specialty` has half of its values missing.

**Target variable:** 30-day remission, i.e. unplanned or unexpected readmission to the same hospital within 30 days of being discharged.

<a id="data-preparation"></a>
## Data preparation


### Removing duplicates

There are a few rows in the data set that refer to the same patient number. Most machine learning models require the observations to be independent from each other. More specifically, the assumption is that observations shouldn't come from repeated measurements, so these will be deleted.

In [10]:
print("df size before:", df.shape[0])
df2 = df.drop_duplicates(subset="patient_nbr", keep="first")
print("df size after:", df2.shape[0])

df size before: 101766
df size after: 71518


### Dropped features

- `encounter_id`, `patient_nbr`, and `payer_code`: irrelevant to patient outcome
- `weight`, `medical_specialty`: too many missing values.
- `admission_source_id`: redundant variable (`admission_type_id` already contains information about admission)
- all medications (except for metformin, glimepiride, glipizide, glyburide, pioglitazone, rosiglitazone, and insulin) too few people using them
- `diag_2` and `diag_ 3`: in the inpatient setting, the primary diagnosis is the most serious and/or resource-intensive during the hospitalization or the inpatient encounter. https://www.hcpro.com/HIM-324035-5707/QA-Primary-principal-and-secondary-diagnoses.html

In [11]:
df3 = df2.drop(["encounter_id",
                "patient_nbr",
                "payer_code",
                "weight", 
                "medical_specialty",
                "admission_source_id",
                "repaglinide",
                "nateglinide",
                "chlorpropamide",
                "acetohexamide",
                "tolbutamide",
                "acarbose",
                "miglitol",
                "troglitazone",
                "tolazamide",
                "examide",
                "citoglipton",
                "glyburide-metformin",
                "glipizide-metformin",
                "glimepiride-pioglitazone",
                "metformin-rosiglitazone",
                "metformin-pioglitazone", 
                "diag_2", 
                "diag_3"], axis=1)

After dropping the unnecessary features, this is our new set of columns:

In [12]:
df3.columns

Index(['race', 'gender', 'age', 'admission_type_id',
       'discharge_disposition_id', 'time_in_hospital', 'num_lab_procedures',
       'num_procedures', 'num_medications', 'number_outpatient',
       'number_emergency', 'number_inpatient', 'diag_1', 'number_diagnoses',
       'max_glu_serum', 'A1Cresult', 'metformin', 'glimepiride', 'glipizide',
       'glyburide', 'pioglitazone', 'rosiglitazone', 'insulin', 'change',
       'diabetesMed', 'readmitted'],
      dtype='object')

#### Age

The age group feature in this data set can be turned into an ordinal variable, thus avoid creating too many new features after the one-hot encoding process.

In [13]:
age_map = {'[0-10)': 0,
           '[10-20)': 1,
           '[20-30)': 2,
           '[30-40)': 3,
           '[40-50)': 4,
           '[50-60)': 5,
           '[60-70)': 6,
           '[70-80)': 7,
           '[80-90)': 8,
           '[90-100)': 9}

df3["age"].replace(age_map, inplace=True)

#### Gender

Gender will be converted to either 0 if male and 1 if female.

In [14]:
df3["gender"] = df3["gender"].apply(lambda x: 1 if x == "Female" else 0)

#### Admission type ID

There are 9 distinct values in this column:

In [15]:
admtype_map

,admission_type_id,description
0,1,Emergency
1,2,Urgent
2,3,Elective
3,4,Newborn
4,5,Not Available
5,6,NaN
6,7,Trauma Center
7,8,Not Mapped


I'll split up this feature between "Emergency" (which will include "Urgent") and "other".

In [16]:
df3["emergency"] = df3["admission_type_id"].apply(lambda x: 1 if x in ["Emergency", "Urgent"] else 0)
df3.drop("admission_type_id", axis=1, inplace=True)

#### Discharge disposition ID

This feature has 29 distinct features, which would negatively impact the model's performance if we create dummy values from them. Therefore, I'm going to restrict these values to either "discharged to home" or other.

In [17]:
discharge_map

,discharge_disposition_id,description
0,1,Discharged to home
1,2,Discharged/transferred to another short term h...
2,3,Discharged/transferred to SNF
3,4,Discharged/transferred to ICF
4,5,Discharged/transferred to another type of inpa...
5,6,Discharged/transferred to home with home healt...
6,7,Left AMA
7,8,Discharged/transferred to home under care of H...
8,9,Admitted as an inpatient to this hospital
9,10,Neonate discharged to another hospital for neo...


In [18]:
df3["discharge_to_home"] = df3["discharge_disposition_id"].apply(lambda x: 1 if x in [1, 6, 8] else 0)
df3.drop("discharge_disposition_id", axis=1, inplace=True)

#### diag_1

The diagnosis features are represented by ICD-9 codes. The idea here is to condensate the hundreds of codes into just a few major groups. 

There are hundreds of different diseases being entered as the `diag`s features, so to avoid ending up with too many features after one-hot encoding them, I'll group them into a few major categories according to the list of ICD-9 codes on Wikipedia: 

    List of ICD-9 codes 001–139: infectious and parasitic diseases
    List of ICD-9 codes 140–239: neoplasms
    List of ICD-9 codes 240–279: endocrine, nutritional and metabolic diseases, and immunity disorders
    List of ICD-9 codes 280–289: diseases of the blood and blood-forming organs
    List of ICD-9 codes 290–319: mental disorders
    List of ICD-9 codes 320–389: diseases of the nervous system and sense organs
    List of ICD-9 codes 390–459: diseases of the circulatory system
    List of ICD-9 codes 460–519: diseases of the respiratory system
    List of ICD-9 codes 520–579: diseases of the digestive system
    List of ICD-9 codes 580–629: diseases of the genitourinary system
    List of ICD-9 codes 630–679: complications of pregnancy, childbirth, and the puerperium
    List of ICD-9 codes 680–709: diseases of the skin and subcutaneous tissue
    List of ICD-9 codes 710–739: diseases of the musculoskeletal system and connective tissue
    List of ICD-9 codes 740–759: congenital anomalies
    List of ICD-9 codes 760–779: certain conditions originating in the perinatal period
    List of ICD-9 codes 780–799: symptoms, signs, and ill-defined conditions
    List of ICD-9 codes 800–999: injury and poisoning
    List of ICD-9 codes E and V codes: external causes of injury and supplemental classification

In [19]:
def icd(code):
    """Converts ICD-9 code in string format into a descriptive major category.
    ---
    code : int
    The ICD-9 encoded diagonisis
    
    return : str
    A major diagnostic category.
    """
    try:
        code = int(code)
    except ValueError:
        return "Other"
    if (code >= 1) and (code < 140):
        return "Infections"
    elif (code >= 140) and (code < 250):
        return "Endocrine"
    elif (code >= 250) and (code < 251):
        return "Diabetes"
    elif (code >= 251) and (code <280):
        return "Endocrine"
    elif (code >= 280) and (code < 290):
        return "Blood"
    elif (code >= 290) and (code < 320):
        return "Mental"
    elif (code >= 320) and (code < 390):
        return "Nervous system"
    elif (code >= 390) and (code < 460):
        return "Circulatory"
    elif (code >= 460) and (code < 520):
        return "Respiratory"
    elif (code >= 520) and (code < 580):
        return "Digestive"
    elif (code >= 580) and (code < 630):
        return "Genitourinary" 
    elif (code >= 630) and (code < 680):
        return "Pregnancy complications"
    elif (code >= 680) and (code < 710):
        return "Skin"
    elif (code >= 710) and (code < 740):
        return "Musculoskeletal"
    else: 
        return "Other"
    
    

In [20]:
df3["diag_1"] = df3["diag_1"].apply(icd)

### Glucose serum test result (max_glu_serum) and A1c test

This variable indicates the range of the result if the test was not taken. It can be converted into an ordinal variable and spare the creation of unnecessary one-hot features. According to the [Mayo Clinic](https://www.mayoclinic.org/diseases-conditions/diabetes/diagnosis-treatment/drc-20371451), a blood sugar level less than 140 mg/dL is normal. A reading between 140 and 199 mg/dL indicates prediabetes. A reading of more than 200 mg/dL after two hours indicates diabetes.

However, the data set isn't clear whether the glucose serum test was made during fasting or at random, so the "official" ranges from the Mayo Clinic may be not be used as a basis for comparison.

According to the literature, adjusting the patient's medication according to HbA1C levels can diminish early readmission rates. Therefore, this new variable is going to be incorporated into the model, and will be 0 if `A1Cresult` is "none" or 1 otherwise. So I want a new variable to check whether a glucose was ordered.

In [21]:
df3["A1C_tested"] = df["A1Cresult"].apply(lambda x: 0 if x == "None" else 1)
df3["max_glu_serum_tested"] = df["max_glu_serum"].apply(lambda x: 0 if x == "None" else 1)

#### Change of medication

Converting object type to numerical:

In [22]:
df3["change"] = df3["change"].apply(lambda x: 1 if x == "Ch" else 0)

#### Diabetes medications
Converting object type to numerical:

In [23]:
df3["diabetesMed"] = df3["diabetesMed"].apply(lambda x: 1 if x == "Yes" else 0)

#### Readmitted (target variable)

In [24]:
df3['readmitted'] = df3['readmitted'].apply(lambda x: 1 if x == "<30" else 0)

#### Dummy variables creation

All categorical variables are going to be converted into dummy variables (one-hot encoding).

In [28]:
df4 = pd.get_dummies(df3, dummy_na=True, drop_first=False)

#### Summary of transformed data set

In [26]:
rf = 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71518 entries, 0 to 101765
Data columns (total 28 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   race                  69570 non-null  object
 1   gender                71518 non-null  int64 
 2   age                   71518 non-null  int64 
 3   time_in_hospital      71518 non-null  int64 
 4   num_lab_procedures    71518 non-null  int64 
 5   num_procedures        71518 non-null  int64 
 6   num_medications       71518 non-null  int64 
 7   number_outpatient     71518 non-null  int64 
 8   number_emergency      71518 non-null  int64 
 9   number_inpatient      71518 non-null  int64 
 10  diag_1                71518 non-null  object
 11  number_diagnoses      71518 non-null  int64 
 12  max_glu_serum         71518 non-null  object
 13  A1Cresult             71518 non-null  object
 14  metformin             71518 non-null  object
 15  glimepiride           71518 non-nul

## Modeling

In [31]:
y = df4['readmitted']
X = df4.drop("readmitted", axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split()

In [30]:
rf = RandomForestClassifier()

## Evaluation